In [4]:
pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/e4/14/d795bb156f8cc10eb1dcfe1332b7dbb8405b634688980aa9be8f885cc888/tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.16.1 from https://files.pythonhosted.org/packages/e0/36/6278e4e7e69a90c00e0f82944d8f2713dd85a69d1add455d9e50446837ab/tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for google-pasta>=0.1.1 from https://files.pythonhosted.org/packages/a3/de/c648ef6835192e6e2cc03f40b19eeda4382c49b5bafb43d88b931c4c74ac/google_pasta-0.2.0-py3-none-any.whl.metadata
  Obtaining depe

# Preprocessing

In [2]:
# import cv2
# import numpy as np
# import os
# import random

# # Function to apply rotation and shear transformations to an image
# def augment_image(image, angle_range=(-15, 15), shear_range=(-0.2, 0.2)):
#     rows, cols, _ = image.shape

#     # Random rotation
#     angle = random.uniform(angle_range[0], angle_range[1])
#     M_rot = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)

#     # Random shear
#     shear_x = random.uniform(shear_range[0], shear_range[1])
#     shear_y = random.uniform(shear_range[0], shear_range[1])
#     M_shear = np.array([[1, shear_x, 0], [shear_y, 1, 0]], dtype=np.float32)

#     # Apply transformations
#     image_rotated = cv2.warpAffine(image, M_rot, (cols, rows))
#     image_sheared = cv2.warpAffine(image_rotated, M_shear, (cols, rows))

#     return image_sheared

# # Paths to the image directories
# root_directory = "C:\\Users\\soura\\Downloads\\picz"  # Set this to your root directory containing subfolders
# output_directory = "C:\\Users\\soura\\Downloads\\picz"  # Set this to where you want to save the augmented images

# if not os.path.exists(output_directory):
#     os.makedirs(output_directory)

# # Process each subfolder
# for subfolder_name in os.listdir(root_directory):
#     subfolder_path = os.path.join(root_directory, subfolder_name)
#     if os.path.isdir(subfolder_path):
#         output_subfolder_path = os.path.join(output_directory, subfolder_name)
#         if not os.path.exists(output_subfolder_path):
#             os.makedirs(output_subfolder_path)
        
#         image_files = [f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))]
        
#         for image_file in image_files:
#             image_path = os.path.join(subfolder_path, image_file)
#             image = cv2.imread(image_path)
            
#             for i in range(1, 11):  # Generate 10 augmented images
#                 augmented_image = augment_image(image)
#                 output_image_path = os.path.join(output_subfolder_path, f'{subfolder_name}_{i}.jpg')
#                 cv2.imwrite(output_image_path, augmented_image)
# print("done")

done


In [2]:
import os
import cv2
import numpy as np
import mediapipe as mp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands

# Function to extract hand keypoints from an image
def extract_keypoints(image):
    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = hands.process(image_rgb)
    keypoints = []
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            keypoints.extend([(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark])
    hands.close()
    return np.array(keypoints).flatten()

# Paths to the image directories
image_directory = "C:\\Users\\soura\\Downloads\\database"  # Update with the correct path
labels = []
data = []

# Load images and extract keypoints
for folder_name in os.listdir(image_directory):
    folder_path = os.path.join(image_directory, folder_name)
    if os.path.isdir(folder_path):
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            image = cv2.imread(image_path)
            keypoints = extract_keypoints(image)
            if keypoints.size > 0:
                data.append(keypoints)
                labels.append(folder_name)

# Find the maximum sequence length
max_length = max(len(seq) for seq in data)

# Pad sequences to ensure they have the same length
data_padded = np.zeros((len(data), max_length))
for i, seq in enumerate(data):
    data_padded[i, :len(seq)] = seq

data = data_padded
labels = np.array(labels)

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels = to_categorical(labels)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.3, random_state=42)


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define the model
model = Sequential([
    LSTM(128, input_shape=(X_train.shape[1], 1), return_sequences=True),
    Dropout(0.2),
    LSTM(128),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Reshape data for LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=40, batch_size=40)


c:\Users\soura\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 14s 360ms/step - accuracy: 0.0343 - loss: 3.3663 - val_accuracy: 0.0674 - val_loss: 3.3548
Epoch 2/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 310ms/step - accuracy: 0.0426 - loss: 3.3461 - val_accuracy: 0.0375 - val_loss: 3.3080
Epoch 3/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 281ms/step - accuracy: 0.0486 - loss: 3.2670 - val_accuracy: 0.0375 - val_loss: 3.2109
Epoch 4/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 304ms/step - accuracy: 0.0621 - loss: 3.2198 - val_accuracy: 0.0449 - val_loss: 3.2196
Epoch 5/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 325ms/step - accuracy: 0.0815 - loss: 3.1842 - val_accuracy: 0.0637 - val_loss: 3.1874
Epoch 6/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 317ms/step - accuracy: 0.0870 - loss: 3.1750 - val_accuracy: 0.0449 - val_loss: 3.1688
Epoch 7/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 338ms/step - accuracy: 0.0708 - loss: 3.1460 - val_accuracy: 0.0637 - val_loss: 3.1572
Epoch 8/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 298ms/step - accuracy: 0.0691 - loss: 3.1630 - val_accuracy: 0

In [4]:
import cv2
import numpy as np
import mediapipe as mp
import time

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Function to predict gesture from keypoints
def predict_gesture(keypoints):
    keypoints = np.array(keypoints).flatten()
    keypoints = keypoints.reshape(1, -1, 1)
    prediction = model.predict(keypoints)
    gesture_id = np.argmax(prediction)
    gesture = label_encoder.inverse_transform([gesture_id])[0]
    return gesture

# List to store the sequence of keypoints
sequence = []
sequence_length = 30  # Adjust as needed
current_gesture = ""

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame to detect hands
    result = hands.process(frame_rgb)
    keypoints_list = []
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            keypoints = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
            keypoints_list.append(keypoints)
            # Draw landmarks on the frame
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
    # Add the keypoints to the sequence
    if keypoints_list:
        sequence.append(keypoints_list[0])  # Taking the first detected hand
    
    # If sequence length exceeds the defined length, remove the oldest entry
    if len(sequence) > sequence_length:
        sequence.pop(0)
    
    # If sequence is long enough, make a prediction
    if len(sequence) == sequence_length:
        print(type(sequence))
        print(sequence[0])
        current_gesture = predict_gesture(sequence)
    
    # Display the predicted gesture on the frame
    cv2.putText(frame, f'Gesture: {current_gesture}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Display the frame
    cv2.imshow('Live Feed', frame)
    
    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
hands.close()


c:\Users\soura\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


<class 'list'>
[(0.04927102476358414, 0.908316433429718, 5.064697461421019e-07), (0.08546648174524307, 0.8867537975311279, -0.04247484356164932), (0.11009178310632706, 0.8414946794509888, -0.05976947024464607), (0.12762604653835297, 0.7975237369537354, -0.06835845857858658), (0.13783584535121918, 0.7561923265457153, -0.0748145803809166), (0.055454254150390625, 0.7594272494316101, -0.04178571328520775), (0.05921875312924385, 0.6825127005577087, -0.0551704540848732), (0.06298606097698212, 0.6314632892608643, -0.06632454693317413), (0.0662279948592186, 0.5925566554069519, -0.07301091402769089), (0.03524453565478325, 0.7531701922416687, -0.022770516574382782), (0.0328301377594471, 0.6746820211410522, -0.035952650010585785), (0.03679630160331726, 0.6165689826011658, -0.047680240124464035), (0.04008547216653824, 0.5748077034950256, -0.054860085248947144), (0.02240140363574028, 0.7536259889602661, -0.005745570175349712), (0.016997013241052628, 0.6812505722045898, -0.017028387635946274), (0.01

c:\Users\soura\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
<class 'list'>
[(0.043547749519348145, 0.9018054008483887, 5.607347475233837e-07), (0.08603538572788239, 0.8712250590324402, -0.03140794113278389), (0.121866375207901, 0.8115331530570984, -0.04436152055859566), (0.14594049751758575, 0.761249840259552, -0.05361802875995636), (0.16353212296962738, 0.7228606343269348, -0.06207043305039406), (0.08048128336668015, 0.7101837396621704, -0.021125957369804382), (0.09292188286781311, 0.6448166370391846, -0.03623282164335251), (0.09721881151199341, 0.5986242294311523, -0.051623452454805374), (0.10014927387237549, 0.5593648552894592, -0.06274900585412979), (0.04506225883960724, 0.6999788284301758, -0.017754266038537025), (0.044461898505687714, 0.6174297332763672, -0.030935021117329597), (0.04475259780883789, 0.5611068606376648, -0.045293740928173065), (0.04541613906621933, 0.5134835243225098, -0.05486760661005974), (0.012812938541173935, 0.7087202668190002, -0.017826560884714127), (0.007543936371803284, 0.629

c:\Users\soura\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step
<class 'list'>
[(0.1272316575050354, 0.8545548319816589, 4.998622671337216e-07), (0.16602830588817596, 0.8115013241767883, -0.02792917937040329), (0.18405523896217346, 0.7535994052886963, -0.039190322160720825), (0.1845429241657257, 0.7009040117263794, -0.050394099205732346), (0.19041316211223602, 0.6523853540420532, -0.05920865759253502), (0.17992569506168365, 0.6556878685951233, -0.013042130507528782), (0.19697017967700958, 0.5806472301483154, -0.03237195312976837), (0.20375442504882812, 0.5320062637329102, -0.04477374255657196), (0.20799410343170166, 0.4909491539001465, -0.05147765949368477), (0.14442555606365204, 0.6586069464683533, -0.0140406284481287), (0.13443325459957123, 0.5731569528579712, -0.038049593567848206), (0.12948451936244965, 0.5195070505142212, -0.053069569170475006), (0.12606674432754517, 0.47206395864486694, -0.058854587376117706), (0.11153271794319153, 0.6831328272819519, -0.018906796351075172), (0.10414798557758331, 0.65939

c:\Users\soura\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
<class 'list'>
[(0.12935574352741241, 0.8551139831542969, 4.2941269384755287e-07), (0.173365980386734, 0.8103548288345337, -0.02451515756547451), (0.19398222863674164, 0.7455782294273376, -0.032621439546346664), (0.1887698620557785, 0.6899245977401733, -0.041514281183481216), (0.2026655077934265, 0.6444946527481079, -0.048448264598846436), (0.1802215576171875, 0.6557677388191223, -0.0076769813895225525), (0.19341033697128296, 0.5802822113037109, -0.02507244050502777), (0.19725137948989868, 0.533035397529602, -0.03550957515835762), (0.19882869720458984, 0.4930477738380432, -0.040887605398893356), (0.14462944865226746, 0.6606118083000183, -0.009507270529866219), (0.13649320602416992, 0.5755981206893921, -0.03207194805145264), (0.1304442584514618, 0.5232589244842529, -0.04560195654630661), (0.12492433190345764, 0.47811001539230347, -0.05005789175629616), (0.11254248023033142, 0.6852619051933289, -0.015144315548241138), (0.10533059388399124, 0.6600578

c:\Users\soura\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
<class 'list'>
[(0.12935574352741241, 0.8551139831542969, 4.2941269384755287e-07), (0.173365980386734, 0.8103548288345337, -0.02451515756547451), (0.19398222863674164, 0.7455782294273376, -0.032621439546346664), (0.1887698620557785, 0.6899245977401733, -0.041514281183481216), (0.2026655077934265, 0.6444946527481079, -0.048448264598846436), (0.1802215576171875, 0.6557677388191223, -0.0076769813895225525), (0.19341033697128296, 0.5802822113037109, -0.02507244050502777), (0.19725137948989868, 0.533035397529602, -0.03550957515835762), (0.19882869720458984, 0.4930477738380432, -0.040887605398893356), (0.14462944865226746, 0.6606118083000183, -0.009507270529866219), (0.13649320602416992, 0.5755981206893921, -0.03207194805145264), (0.1304442584514618, 0.5232589244842529, -0.04560195654630661), (0.12492433190345764, 0.47811001539230347, -0.05005789175629616), (0.11254248023033142, 0.6852619051933289, -0.015144315548241138), (0.10533059388399124, 0.6600578